# Label modifcation by Kamiran et al. - Adult data

This notebook contains the implementation of the post-processing label modification algorithm introduced in [Decision Theory for Discrimination-aware Classification](https://ieeexplore.ieee.org/abstract/document/6413831) by Kamiran et al. (2012) as part of the IBM AIF360 fairness tool box github.com/IBM/AIF360.

The intervention method identifies a low-confidence subset of the data for probabilistic classifiers, and produces new predicted labels for this subset by assigning positive labels to the unprivileged group and negative labels to the privileged group. Different notions of fairness can be achieved in this way, including demographic parity, equalised odds and equal opportunity, of which each one is allowed in the implementation. In this notebook we demonstrate each of these three definitions as a result of the fairness intervention. 

For simplicity, we'll focus mitigating bias with resepct to sex.

In [ ]:
from pathlib import Path

import joblib
import pandas as pd
import plotly.graph_objs as go
from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing.reject_option_classification import (
    RejectOptionClassification,
)
from helpers.fairness_measures import (
    accuracy,
    disparate_impact_d,
    equalised_odds_d,
    equal_opportunity_d,
)
from helpers.finance import preprocess

In [ ]:
from helpers import export_plot

Location of artifacts (model and data)

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "adult"
preprocess(data_dir)

## Load data

Location of data

In [ ]:
train = pd.read_csv(data_dir / "processed" / "train-one-hot.csv").sample(6000)
val = pd.read_csv(data_dir / "processed" / "val-one-hot.csv").sample(2000)
test = pd.read_csv(data_dir / "processed" / "test-one-hot.csv").sample(2000)

In order to process data for our fairness intervention we need to define special dataset objects which are part of every intervention pipeline within the IBM AIF360 toolbox. These objects contain the original data as well as some useful further information, e.g., which feature is the protected attribute as well as which column corresponds to the label.

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)

Define which binary value goes with the (un-)privileged group

In [ ]:
privileged_groups = [{"sex": 1.0}]
unprivileged_groups = [{"sex": 0.0}]

## Load original model

We introduce a baseline unfair model, which is trained for performance only and to which the post-processing intervention is applied.

In [ ]:
# baseline_model = MLPClassifier(
#     hidden_layer_sizes=(100, 100), early_stopping=True
# )
# baseline_model.fit(train.drop("salary", axis=1), train.salary)

For maximum reproducibility we can also load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [ ]:
baseline_model = joblib.load(
    artifacts_dir / "models" / "finance" / "baseline.pkl"
)

Get predictions for the validation and test data

In [ ]:
bl_val_scores = baseline_model.predict_proba(val.drop("salary", axis=1))[:, 1]
val_sds_pred = val_sds.copy(deepcopy=True)
val_sds_pred.scores = bl_val_scores.reshape(-1, 1)

bl_test_scores = baseline_model.predict_proba(test.drop("salary", axis=1))[
    :, 1
]
test_sds_pred = test_sds.copy(deepcopy=True)
test_sds_pred.scores = bl_test_scores.reshape(-1, 1)

## Demographic parity

We first address demographic parity. In order to do so, we learn the label modification algorithm based on the true and predicted labels of the validation data. We then apply the learnt intervention to the predictions of the test data and analyse the outcomes for fairness and accuracy.

In [ ]:
# Metric used
metric_name = "Statistical parity difference"  # Alias for demographic parity

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05

### Learn intervention

On validation data.

In [ ]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)
ROC = ROC.fit(val_sds, val_sds_pred)

### Apply intervention

On test data.

In [ ]:
# Transform the test set
test_sds_pred_transf = ROC.predict(test_sds_pred).copy(deepcopy=True)
test_pred_labels = test_sds_pred_transf.labels.flatten()

### Analyse fairness and accuracy

On test data.

In [ ]:
test_features = test.drop(columns=["sex", "salary"]).values
test_sex = test[["sex"]].values.flatten()
test_salary = test["salary"].values.flatten()
mask = test_sex == 1

# baseline metrics
bl_test_acc = accuracy(bl_test_scores, test_salary)
bl_test_did = disparate_impact_d(bl_test_scores, test_sex)

# new model metrics
test_acc = accuracy(test_pred_labels, test_salary)
test_did = disparate_impact_d(test_pred_labels, test_sex)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline demographic parity (dist.): {bl_test_did:.3f}")
print(f"Demographic parity (dist.): {test_did:.3f}\n")

Consider accuracy on the female / male subgroups

In [ ]:
print(
    "Female accuracy =",
    accuracy(test_pred_labels[test.sex == 0], test.salary[test.sex == 0]),
)
print(
    "Male accuracy =",
    accuracy(test_pred_labels[test.sex == 1], test.salary[test.sex == 1]),
)
print("Mean female score =", test_pred_labels[test.sex == 0].mean())
print("Mean male score =", test_pred_labels[test.sex == 1].mean())

In [ ]:
dp_bar = go.Figure(
    data=[
        go.Bar(
            x=[sex],
            y=[test_pred_labels[test_sex == sex].mean()],
            name="Male" if sex else "Female",
        )
        for sex in range(2)
    ]
)
dp_bar

In [ ]:
export_plot(dp_bar, "kamiran-dp.json")

## Equalised Odds

We'll now repeat the process for equalised odds, which requires us changing the underlying metric which leads to learning the correction according a new decision threshold addressing equalised odds. There are no further modifcations to the existing parameter choices required. However we apply smaller bounds on equalised odds than for demographic parity.

In [ ]:
# Metric used
metric_name = "Average odds difference"  # alias for equalised odds

# Upper and lower bound on the fairness metric used
metric_ub = 0.01
metric_lb = -0.01

### Learn intervention
On validation data.

In [ ]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)
ROC = ROC.fit(val_sds, val_sds_pred)

### Apply intervention
On test data.

In [ ]:
# Transform the test set
test_sds_pred_transf = ROC.predict(test_sds_pred).copy(deepcopy=True)
test_pred_labels = test_sds_pred_transf.labels.flatten()

### Analyse fairness and accuracy

On test data.

In [ ]:
test_features = test.drop(columns=["sex", "salary"]).values
test_sex = test[["sex"]].values.flatten()
test_salary = test["salary"].values.flatten()
mask = test_sex == 1

# baseline metrics
bl_test_acc = accuracy(bl_test_scores, test_salary)
bl_test_eod = equalised_odds_d(bl_test_scores, test_sex, test_salary)

# new model metrics
test_acc = accuracy(test_pred_labels, test_salary)
test_eod = equalised_odds_d(test_pred_labels, test_sex, test_salary)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equalised odds (dist.): {bl_test_eod:.3f}")
print(f"Equalised odds (dist.): {test_eod:.3f}\n")

In [ ]:
eo_bar = go.Figure(
    data=[
        go.Bar(
            x=[label],
            y=[
                test_pred_labels[
                    (test.sex == sex) & (test.salary == label)
                ].mean()
            ],
            name="Male" if sex else "Female",
        )
        for label in range(2)
        for sex in range(2)
    ]
)

eo_bar

In [ ]:
export_plot(eo_bar, "kamiran-eo.json")

## Equal opportunity
We'll now repeat the process for equalised opportunity, which only requires us changing the underlying metric.

In [ ]:
# Metric used
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.01
metric_lb = -0.01

### Learn interrvention
On validation data.

In [ ]:
ROC = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name=metric_name,
    metric_ub=metric_ub,
    metric_lb=metric_lb,
)
ROC = ROC.fit(val_sds, val_sds_pred)

### Apply intervention
On test data.

In [ ]:
# Transform the test set
test_sds_pred_transf = ROC.predict(test_sds_pred).copy(deepcopy=True)
test_pred_labels = test_sds_pred_transf.labels.flatten()

### Analyse fairness and accuracy

On test data.

In [ ]:
test_features = test.drop(columns=["sex", "salary"]).values
test_sex = test[["sex"]].values.flatten()
test_salary = test["salary"].values.flatten()
mask = test_sex == 1

# baseline metrics
bl_test_acc = accuracy(bl_test_scores, test_salary)
bl_test_eoppd = equal_opportunity_d(bl_test_scores, test_sex, test_salary)

# new model metrics
test_acc = accuracy(test_pred_labels, test_salary)
test_eoppd = equal_opportunity_d(test_pred_labels, test_sex, test_salary)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equalised odds (dist.): {bl_test_eoppd:.3f}")
print(f"Equalised odds (dist.): {test_eoppd:.3f}\n")

In [ ]:
eopp_bar = go.Figure(
    data=[
        go.Bar(
            x=[sex],
            y=[
                test_pred_labels[
                    (test.sex == sex) & (test.salary == 1.0)
                ].mean()
            ],
            name="Male" if sex else "Female",
        )
        for sex in range(2)
    ]
)
eopp_bar

In [ ]:
export_plot(eopp_bar, "kamiran-eopp.json")